In [1]:
# import necessary dependencies
from itertools import product as itp
from pythtb import *
from triqs.lattice.tight_binding import TBLattice
import sympy as sp
import warnings
import numpy as np
import matplotlib.pyplot as plt

Starting serial run at: 2023-08-03 14:58:09.294450


In [2]:
from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('La2CuO4 Files', 'lco')
fermi_ev = 12.7367
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.038, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)

In [4]:
def compareBandStructure(w90_input, fermi_ev):
    # get band structure from Wannier90 in the form of
    # list of k-points in reduced coordinates
    # energies interpolated by Wannier90
    (w90_kpt, w90_evals) = w90_input.w90_bands_consistency()

    # obtain simplified models based on various cutoffs
    w90_model_038 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.038, max_distance = None)
    w90_model_05 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.05, max_distance = None)
    w90_model_45 = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.45, max_distance = None)

    # solve simplified models on the same k-path as in Wannier90
    int_evals_038 = w90_model_038.solve_all(w90_kpt)
    int_evals_05 = w90_model_05.solve_all(w90_kpt)
    int_evals_45 = w90_model_45.solve_all(w90_kpt)

    # setting up the plots
    fig, ax = plt.subplots(dpi = 2000)

    # plotting the full Wannier90 model
    for i in range(w90_evals.shape[0]):
        x_axis = list(range(w90_evals.shape[1]))
        y_axis = w90_evals[i] - fermi_ev
        ax.plot(x_axis, y_axis, "k-", zorder = -100)

    # plotting the minimal models
    for i in range(int_evals_038.shape[0]):
        ax.plot(list(range(int_evals_038.shape[1])), int_evals_038[i], "r-", zorder = -50)
        ax.plot(list(range(int_evals_05.shape[1])), int_evals_05[i], "g-", zorder = -50)
        ax.plot(list(range(int_evals_45.shape[1])), int_evals_45[i], "c-", zorder = -50)
    
    # plotting the horizontal line that passes through the y-axis
    ax.axhline(y = 0, color = 'm', linestyle = '--')
    ax.set_yticks(np.arange(-2, 2.1, step = 1))

    # increasing the width of the curves
    lines = ax.lines
    for line in lines:
        line.set_linewidth(2.5)
    
    # coordinates of the high symmetry points
    G = np.array([0.00, 0.00, 0.00])
    N = np.array([0.25, -0.25, 0.25])
    X = np.array([0.00, 0.00, 0.50])
    M = np.array([0.25, 0.25, -0.25])

    # x-axis labels of the high-symmetry points
    kpath_labels = ['G', 'N', 'X', 'G', 'M']
    idx_kpt = {}

    # checking k-point coordinates equal to the high-symmetry point coordinates
    for name, idx in (zip(('G', 'N', 'X', 'M'), [G, N, X, M])):
        idx_kpt[name] = list(np.where((w90_kpt == idx).all(axis = 1))[0])
    
    kpath_xticks = list([x[0] for x in idx_kpt.values()])
    kpath_xticks.append(idx_kpt['G'][1])
    kpath_xticks = sorted(kpath_xticks)
    ax.set_xticks(kpath_xticks)
    ax.set_xticklabels(kpath_labels)


    for n in range(len(kpath_xticks)):
        ax.axvline(x = kpath_xticks[n], linewidth = 1, color = 'k')

    
    ax.set_xlim(0, int_evals_038.shape[1] - 1)
    ax.set_ylabel(r"Band Energy - $E_F$ (eV)")
    ax.set_title("Comparing Minimal Models")
    ax.legend(["Full model", "Cutoff at 0.038 eV", "Cutoff at 0.05 eV", "Cutoff at 0.45 eV"])

# calling the function
lanthanum_w90_input = w90('La2CuO4 Files', 'lco')
compareBandStructure(lanthanum_w90_input, 12.7367)